In [10]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


# 1. Indexing: Load

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [3]:
len(docs[0].page_content)

42824

In [4]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


# 2. Indexing: Split

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200, add_start_index = True
)
all_splits = text_splitter.split_documents(docs)

In [6]:
len(all_splits)

66

In [7]:
len(all_splits[0].page_content)

969

In [8]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

# 3. Indexing: Store

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents = all_splits, embedding = OpenAIEmbeddings())

# 4. Retrieval and Generation: Retrieve

In [12]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs ={"k": 6})

In [17]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

In [19]:
len(retrieved_docs)

6

In [20]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


# 5. Retrieval and Generation: Generate

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-3.5-turbo-0125", temperature=0)

### Using RAG below

In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [24]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [26]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end = "", flush = True)

Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks by thinking step by step or exploring multiple reasoning possibilities. Task decomposition can be done using simple prompting, task-specific instructions, or human inputs.

In [36]:
for chunk in rag_chain.stream("What are the types of memory?"):
    print(chunk, end = "", flush = True)

The types of memory are Short-Term Memory (STM) or Working Memory and Long-Term Memory (LTM). Short-term memory stores information needed for complex cognitive tasks and has a limited capacity and duration. Long-term memory can store information for a long time and has two subtypes: explicit/declarative memory and implicit/procedural memory.